In [2]:
import acquire as get
import pandas as pd
import numpy as np
import describe as tatl
import wrangle as get
from collections import Counter

from sklearn.model_selection import train_test_split
# import scaling methods
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer
# import modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import explained_variance_score



In [7]:
df = get.clean_df()
time_df = get.create_time_df(df)

In [8]:
type(df)
type(time_df)

pandas.core.frame.DataFrame

In [9]:
time_df

,countrycode,country,currency_unit,rgdpe,rgdpo,pop,emp,avh,hc,ccon,...,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,both_sexes,female,male
year,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,CAN,Canada,Canadian Dollar,1.276876e+06,1.284667e+06,30.588383,14.952766,1786.186367,3.504107,8.876788e+05,...,0.581162,0.505927,0.753662,0.513459,0.514761,0.463148,0.808440,10.7,5.0,16.6
2000-01-01 01:00:00,CHN,China,Yuan Renminbi,5.514430e+06,5.420106e+06,1290.550765,735.723144,2090.475311,2.313537,3.939191e+06,...,0.220126,0.280168,0.148072,0.447197,0.482951,0.197761,0.560159,14.9,14.5,15.5
2000-01-01 02:00:00,CZE,Czech Republic,Czech Koruna,2.288185e+05,2.315232e+05,10.289373,4.886872,1895.562155,3.573689,1.670260e+05,...,0.278533,0.341834,0.218788,0.469314,0.457078,0.271949,0.357402,13.4,5.1,22.6
2000-01-01 03:00:00,DEU,Germany,Euro,3.030253e+06,3.035472e+06,81.400882,39.603115,1465.937805,3.566319,2.240533e+06,...,0.606525,0.648399,0.847104,0.515353,0.536459,0.646678,0.854531,11.2,5.3,17.6
2000-01-01 04:00:00,FIN,Finland,Euro,1.860555e+05,1.865058e+05,5.187954,2.305018,1704.713453,3.131667,1.161209e+05,...,0.694162,0.593000,0.839391,0.504715,0.486676,0.561708,1.085940,21.7,10.4,33.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01 10:00:00,LTU,Lithuania,Euro,1.034802e+05,8.964218e+04,2.759627,1.379529,1886.084474,3.296741,8.059303e+04,...,0.556210,0.549069,0.433854,0.657423,0.548743,0.460375,0.696666,20.2,6.2,36.1
2019-01-01 11:00:00,POL,Poland,Zloty,1.259694e+06,1.211846e+06,37.887768,16.159107,2022.632232,3.457758,9.740322e+05,...,0.489376,0.523544,0.392328,0.683009,0.617757,0.420247,0.786552,9.3,2.4,16.5
2019-01-01 12:00:00,SGP,Singapore,Singapore Dollar,5.143763e+05,4.779079e+05,5.804337,3.759604,2330.165875,4.351568,2.296615e+05,...,0.761578,0.687142,0.709089,0.711649,0.673213,0.469200,0.894616,9.7,6.4,12.7


In [21]:
time_train = time_df[:'2013']
time_test = time_df['2014':]


In [22]:
time_train.shape

(210, 47)

In [23]:
time_test.shape

(90, 47)

In [13]:
time_df.shape

(300, 47)

In [24]:
210/300

0.7

In [27]:
def split_time_with_val(df):
    '''
   Using methods gleaned from Codeup's Time Series Analysis lessons,
   splits the Superstore DF into Train Validate and Split; .5/.3/.2 respectively.
   Subsequently returns each.
    '''
    print('Dataframe Input received: Splitting Data .5/.3/.2.')
    train_size = int(len(df) * .5)
    validate_size = int(len(df) * .3)
    test_size = int(len(df) - train_size - validate_size)
    validate_end_index = train_size + validate_size

    # split into train, validation, test
    train_time = df[: train_size]
    validate_time = df[train_size : validate_end_index]
    test_time = df[validate_end_index : ]
    print(f'Train: {train_time.shape}, Validate {validate_time.shape}, and Test {test_time.shape} are ready.\
    \n Proceed with EDA.')
    return train_time, validate_time, test_time

In [28]:
train_time, validate_time, test_time = split_time_with_val(time_df)

Dataframe Input received: Splitting Data .5/.3/.2.
Train: (150, 47), Validate (90, 47), and Test (60, 47) are ready.    
 Proceed with EDA.


In [40]:
test_time

,countrycode,country,currency_unit,rgdpe,rgdpo,pop,emp,avh,hc,ccon,...,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,both_sexes,female,male
year,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,CAN,Canada,Canadian Dollar,1.683514e+06,1.711195e+06,36.382944,18.541105,1705.192385,3.698929,1.306512e+06,...,0.913157,0.855560,0.981499,0.622976,0.658081,0.730589,0.628558,10.4,5.5,15.3
2016-01-01 01:00:00,CHN,China,Yuan Renminbi,1.861120e+07,1.859171e+07,1414.049351,798.530334,2167.227690,2.623665,1.010907e+07,...,0.590271,0.598280,0.668698,0.645273,0.667347,0.402086,0.601590,7.0,5.1,9.0
2016-01-01 02:00:00,CZE,Czech Republic,Czech Koruna,3.911239e+05,3.553079e+05,10.618857,5.302703,1778.525540,3.661578,2.596158e+05,...,0.544972,0.514171,0.418386,0.663135,0.593381,0.408040,0.421572,10.3,4.1,16.9
2016-01-01 03:00:00,DEU,Germany,Euro,4.039049e+06,4.048523e+06,82.193768,42.927361,1394.754324,3.668015,2.841749e+06,...,0.853037,0.784880,0.989583,0.670594,0.657799,0.729840,0.664264,8.8,4.3,13.5
2016-01-01 04:00:00,FIN,Finland,Euro,2.453515e+05,2.340793e+05,5.497713,2.536312,1606.303250,3.447266,1.845267e+05,...,1.000203,0.896610,1.058104,0.654145,0.602945,0.785023,0.802633,13.3,6.1,20.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01 10:00:00,LTU,Lithuania,Euro,1.034802e+05,8.964218e+04,2.759627,1.379529,1886.084474,3.296741,8.059303e+04,...,0.556210,0.549069,0.433854,0.657423,0.548743,0.460375,0.696666,20.2,6.2,36.1
2019-01-01 11:00:00,POL,Poland,Zloty,1.259694e+06,1.211846e+06,37.887768,16.159107,2022.632232,3.457758,9.740322e+05,...,0.489376,0.523544,0.392328,0.683009,0.617757,0.420247,0.786552,9.3,2.4,16.5
2019-01-01 12:00:00,SGP,Singapore,Singapore Dollar,5.143763e+05,4.779079e+05,5.804337,3.759604,2330.165875,4.351568,2.296615e+05,...,0.761578,0.687142,0.709089,0.711649,0.673213,0.469200,0.894616,9.7,6.4,12.7


In [30]:
def split_suicide(df):
    
    df = df.copy()
    # train/validate/test split
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
    
    print(train.shape, validate.shape, test.shape)
    
    
    return df, train, validate, test

In [33]:
df, train, validate, test = split_suicide(df)

(168, 48) (72, 48) (60, 48)


In [34]:
train
# this is a bad idea. I'll need to split based on time but just keep resetting the index as needed
# The randomness of the arrangement may actually have some utility, but none that I can argue for right now. 

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,both_sexes,female,male
110,FRA,France,Euro,2010,2.613961e+06,2.591608e+06,64.952503,26.876162,1539.803062,3.056434,...,1.001260,0.880837,1.186045,0.671687,0.662727,0.851126,0.810859,13.9,7.2,21.2
255,SGP,Singapore,Singapore Dollar,2015,4.514764e+05,4.213812e+05,5.592152,3.655490,2362.423602,3.657473,...,0.736810,0.623167,0.652771,0.675785,0.637721,0.435302,0.926200,7.6,5.0,10.0
219,LTU,Lithuania,Euro,2019,1.034802e+05,8.964218e+04,2.759627,1.379529,1886.084474,3.296741,...,0.556210,0.549069,0.433854,0.657423,0.548743,0.460375,0.696666,20.2,6.2,36.1
128,IND,India,Indian Rupee,2008,4.251417e+06,4.544137e+06,1200.669765,460.812927,2099.633091,1.921647,...,0.264972,0.345728,0.257934,0.666305,0.667793,0.221943,0.473667,16.3,13.1,19.5
228,POL,Poland,Zloty,2008,7.928244e+05,7.907710e+05,38.356786,15.663867,2069.599985,3.185021,...,0.682373,0.816383,0.540947,0.694554,0.643363,0.726797,1.475367,14.4,3.8,25.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,CAN,Canada,Canadian Dollar,2008,1.500648e+06,1.620558e+06,33.337638,17.258270,1740.305980,3.624348,...,1.026286,0.912104,1.188864,0.649821,0.678588,0.965512,1.131241,9.9,5.1,14.7
208,LTU,Lithuania,Euro,2008,7.532916e+04,7.075892e+04,3.212865,1.433779,1933.571514,3.119289,...,0.649136,0.758377,0.503224,0.674347,0.582031,0.711788,1.262024,32.6,9.4,59.0
78,DEU,Germany,Euro,2018,4.275108e+06,4.263030e+06,83.124418,44.306030,1389.931779,3.672922,...,0.917328,0.866180,1.042328,0.723426,0.696419,0.842312,0.654573,8.5,4.0,13.1
229,POL,Poland,Zloty,2009,8.330419e+05,8.183836e+05,38.351916,15.738270,2054.000025,3.207643,...,0.538588,0.645690,0.433900,0.652941,0.589510,0.554199,1.205102,16.1,4.3,28.7


In [3]:
df, train, validate, test = get.wrangle_df()

Dataframe Input received: Splitting Data .5/.3/.2.
Train: (150, 47), Validate (90, 47), and Test (60, 47) are ready.    
 Proceed with EDA.


In [5]:
train

,countrycode,country,currency_unit,rgdpe,rgdpo,pop,emp,avh,hc,ccon,...,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,both_sexes,female,male
year,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,CAN,Canada,Canadian Dollar,1.276876e+06,1.284667e+06,30.588383,14.952766,1786.186367,3.504107,8.876788e+05,...,0.581162,0.505927,0.753662,0.513459,0.514761,0.463148,0.808440,10.7,5.0,16.6
2000-01-01 01:00:00,CHN,China,Yuan Renminbi,5.514430e+06,5.420106e+06,1290.550765,735.723144,2090.475311,2.313537,3.939191e+06,...,0.220126,0.280168,0.148072,0.447197,0.482951,0.197761,0.560159,14.9,14.5,15.5
2000-01-01 02:00:00,CZE,Czech Republic,Czech Koruna,2.288185e+05,2.315232e+05,10.289373,4.886872,1895.562155,3.573689,1.670260e+05,...,0.278533,0.341834,0.218788,0.469314,0.457078,0.271949,0.357402,13.4,5.1,22.6
2000-01-01 03:00:00,DEU,Germany,Euro,3.030253e+06,3.035472e+06,81.400882,39.603115,1465.937805,3.566319,2.240533e+06,...,0.606525,0.648399,0.847104,0.515353,0.536459,0.646678,0.854531,11.2,5.3,17.6
2000-01-01 04:00:00,FIN,Finland,Euro,1.860555e+05,1.865058e+05,5.187954,2.305018,1704.713453,3.131667,1.161209e+05,...,0.694162,0.593000,0.839391,0.504715,0.486676,0.561708,1.085940,21.7,10.4,33.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-01 10:00:00,LTU,Lithuania,Euro,6.538190e+04,6.099171e+04,3.167265,1.319391,1862.943692,3.134434,5.874300e+04,...,0.612641,0.679582,0.458383,0.633214,0.551668,0.598117,0.942484,33.1,9.3,60.6
2009-01-01 11:00:00,POL,Poland,Zloty,8.330419e+05,8.183836e+05,38.351916,15.738270,2054.000025,3.207643,6.935582e+05,...,0.538588,0.645690,0.433900,0.652941,0.589510,0.554199,1.205102,16.1,4.3,28.7
2009-01-01 12:00:00,SGP,Singapore,Singapore Dollar,3.367912e+05,2.916328e+05,4.966614,2.991792,2440.514338,2.974092,1.561003e+05,...,0.642015,0.499609,0.509354,0.652682,0.594564,0.383438,0.863112,9.0,5.4,12.6
